# Renforcement

In [1]:
### Import Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
import locale
import dateparser
import torch
from torch import tensor
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

In [2]:
df_p = pd.read_csv("df_cleaned.csv")

df_p.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20 03:16:24+00:00,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas
1,Florian T,1,2.0,2023-10-19 13:10:44+00:00,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas
2,Anca Visdei,5,1.0,2023-10-19 15:41:51+00:00,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas
3,Pat perso,1,5.0,2023-10-18 16:13:38+00:00,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas
4,Sylvia FABRE,2,4.0,2023-10-17 22:22:10+00:00,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas


In [3]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168806 entries, 0 to 168805
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user             168806 non-null  object 
 1   etoiles          168806 non-null  int64  
 2   n_avis           168806 non-null  float64
 3   date_avis        168806 non-null  object 
 4   titre_avis       168806 non-null  object 
 5   text_avis        168806 non-null  object 
 6   date_experience  168806 non-null  object 
 7   Société          168806 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.3+ MB


### Remettre en datetime

In [4]:
### Import CSV
df_p = pd.read_csv("df_cleaned.csv")


### Retirer les minutes / secondes / décalage

df_p["date_avis"] = df_p["date_avis"].str.split(" ").str[0]
df_p["date_experience"] = df_p["date_experience"].str.split(" ").str[0]

### Transformation en datetime

df_p["date_avis"] = pd.to_datetime(df_p["date_avis"], format='%Y-%m-%d')
df_p["date_experience"] = pd.to_datetime(df_p["date_experience"], format='%Y-%m-%d')

df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168806 entries, 0 to 168805
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   user             168806 non-null  object        
 1   etoiles          168806 non-null  int64         
 2   n_avis           168806 non-null  float64       
 3   date_avis        168806 non-null  datetime64[ns]
 4   titre_avis       168806 non-null  object        
 5   text_avis        168806 non-null  object        
 6   date_experience  168806 non-null  datetime64[ns]
 7   Société          168806 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 10.3+ MB


### Ajouter les colonnes jour / mois / heure / année et les catégoriser + rajouter longueur des textes

In [5]:
### Ajout des colonnes jour / mois / heure / année pour avis

df_p["jour_avis"] = df_p["date_avis"].dt.day
df_p["mois_avis"] = df_p["date_avis"].dt.month
df_p["heure_avis"] = df_p["date_avis"].dt.hour
df_p["annee_avis"] = df_p["date_avis"].dt.year

df_p["jour_experience"] = df_p["date_experience"].dt.day
df_p["mois_experience"] = df_p["date_experience"].dt.month
df_p["heure_experience"] = df_p["date_experience"].dt.hour
df_p["annee_experience"] = df_p["date_experience"].dt.year

### Catégoriser en jour et nuit

df_p["periode_avis"] = df_p["heure_avis"].apply(lambda x: "jour" if 6 <= x <= 22 else "nuit")


df_p["longueur_texte"] = df_p["text_avis"].apply(len)

df_p.head()


,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société,jour_avis,mois_avis,heure_avis,annee_avis,jour_experience,mois_experience,heure_experience,annee_experience,periode_avis,longueur_texte
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas,20,10,0,2023,19,10,0,2023,nuit,460
1,Florian T,1,2.0,2023-10-19,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas,19,10,0,2023,19,10,0,2023,nuit,626
2,Anca Visdei,5,1.0,2023-10-19,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas,19,10,0,2023,19,10,0,2023,nuit,812
3,Pat perso,1,5.0,2023-10-18,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas,18,10,0,2023,17,10,0,2023,nuit,598
4,Sylvia FABRE,2,4.0,2023-10-17,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas,17,10,0,2023,17,10,0,2023,nuit,804


### Chargement et préparation des données pour CamemBert + préprocessing additionnel

In [6]:
### Nettoyage

df_p["text_avis"] = df_p["text_avis"].str.replace("[^\w\s]", "")

### No maj

df_p["text_avis"] = df_p["text_avis"].str.lower()

### Suppression espaces inutiles

df_p["text_avis"] = df_p["text_avis"].str.strip()



### Etoiles pour sentiment

In [7]:
def classify_sentiment(etoiles):
    if etoiles >= 4:
        return "positif"
    elif etoiles == 3:
        return "neutre"
    else:
        return "négatif"

df_p["sentiment_etoiles"] = df_p["etoiles"].apply(classify_sentiment)

### Sauvegarde Data Prime avant analyse sentiments CamemBert

In [8]:
df_p.to_csv("df_prime.csv", encoding="utf-8", index=False)

In [26]:
### Séparation en ensemble de test

X = df_p["text_avis"]
y = df_p["etoiles"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
df_p.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société,jour_avis,mois_avis,heure_avis,annee_avis,jour_experience,mois_experience,heure_experience,annee_experience,periode_avis,longueur_texte,sentiment_etoiles
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20,Orgueil gestionnaire,"ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas,20,10,0,2023,19,10,0,2023,nuit,460,négatif
1,Florian T,1,2.0,2023-10-19,Mauvaise expérience en agence physique et en l...,"mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas,19,10,0,2023,19,10,0,2023,nuit,626,négatif
2,Anca Visdei,5,1.0,2023-10-19,"Chère agence ,","chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas,19,10,0,2023,19,10,0,2023,nuit,812,positif
3,Pat perso,1,5.0,2023-10-18,Mépris ou incompétence ou les 2,"rv pris à l'agence du touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas,18,10,0,2023,17,10,0,2023,nuit,598,négatif
4,Sylvia FABRE,2,4.0,2023-10-17,Pas d'imprimé prévu pour transferts de compte ...,pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas,17,10,0,2023,17,10,0,2023,nuit,804,négatif


In [28]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168806 entries, 0 to 168805
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   user               168806 non-null  object        
 1   etoiles            168806 non-null  int64         
 2   n_avis             168806 non-null  float64       
 3   date_avis          168806 non-null  datetime64[ns]
 4   titre_avis         168806 non-null  object        
 5   text_avis          168806 non-null  object        
 6   date_experience    168806 non-null  datetime64[ns]
 7   Société            168806 non-null  object        
 8   jour_avis          168806 non-null  int32         
 9   mois_avis          168806 non-null  int32         
 10  heure_avis         168806 non-null  int32         
 11  annee_avis         168806 non-null  int32         
 12  jour_experience    168806 non-null  int32         
 13  mois_experience    168806 non-null  int32   

### Tentative 1 

In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

sample = next(iter(train_dataloader))
print(sample)



[tensor([[    5,  5061,     7,  ...,     1,     1,     1],
        [    5,  5061,   662,  ...,     1,     1,     1],
        [    5,    95,   445,  ...,     1,     1,     1],
        ...,
        [    5, 26015,   924,  ...,     1,     1,     1],
        [    5,   424,  1630,  ...,     1,     1,     1],
        [    5,   895, 11508,  ...,     1,     1,     1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([5, 5, 5, 5, 2, 5, 5, 5, 1, 5, 4, 2, 5, 5, 2, 1, 5, 4, 5, 1, 1, 5, 4, 5,
        1, 2, 3, 5, 1, 1, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 1, 5, 3, 2, 5, 5, 1, 5,
        5, 5, 5, 1, 2, 1, 1, 2, 5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 5, 3, 5, 1, 4, 5, 5, 1, 5, 1, 4, 4, 1,
        5, 5, 5, 5, 5, 1, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 1, 5, 5, 5,
        4, 5, 5, 

In [33]:

### Ne pas remplir mon SSD

os.environ["TRANSFORMERS_CACHE"] = "D:\DataScientest\CacheTransformers"

### Chargement de CamemBert

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=1)

### Tokenization

train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

### Dataloader

train_dataset = torch.utils.data.TensorDataset(tensor(train_encodings.input_ids), tensor(train_encodings.attention_mask), tensor(y_train.values))
test_dataset = torch.utils.data.TensorDataset(tensor(test_encodings.input_ids), tensor(test_encodings.attention_mask), tensor(y_test.values))

### Fonction et optimizer

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)

### Train

model.train()
num_epochs = 3

for epoch in range(num_epochs):
    for i, (input_ids, attention_mask, labels) in enumerate(train_dataset):
        optimizer.zero_grad()
        
        # Add batch dimension
        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)
        labels = labels.unsqueeze(0)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze(-1)
        
        loss = criterion(logits, labels.float())
        loss.backward()
        
        optimizer.step()
        if i % 100 == 0:
            print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")

### Test pred

model.eval()
predictions = []

for input_ids, attention_mask, _ in test_dataset:
    with torch.no_grad():
        
        ### tupple
        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        pred = torch.sigmoid(logits).numpy() ### Converstion en proba
        predictions.extend(pred)

save_path = "D:\DataScientest\Models\CamemBert"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Iteration: 0, Loss: 1.2314907312393188
Epoch: 0, Iteration: 100, Loss: -6.4182586669921875
Epoch: 0, Iteration: 200, Loss: 2.3841855067985307e-07
Epoch: 0, Iteration: 300, Loss: -89.72496795654297
Epoch: 0, Iteration: 400, Loss: -111.9465103149414
Epoch: 0, Iteration: 500, Loss: -131.34703063964844
